In [67]:
import pandas as pd
import numpy as np
import itertools
import pymysql
import gensim
import random
import spacy
import re


from bs4 import BeautifulSoup
from tqdm import tqdm
from nltk.corpus import stopwords
from gensim import corpora
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from scipy.sparse import coo_matrix
from sklearn.utils.extmath import safe_sparse_dot
from scipy.spatial.distance import correlation

In [2]:
db_info = {'HOST':'yen-6740db.czewfafbu0y5.us-east-1.rds.amazonaws.com',
           'PORT':3306,
           'USER':'fodkastir',
           'PASSWD':'fodka1234stir',
           'DB':'beer_db',}
def get_conn(db_info):
    conn = pymysql.connect(host=db_info['HOST'],
                           port=db_info['PORT'],
                           user=db_info['USER'],
                           passwd=db_info['PASSWD'],
                           db=db_info['DB'])
    conn.set_charset('utf8')
    return conn

def init_cur(conn):
    cur = conn.cursor()
    cur.execute('SET NAMES utf8;')
    cur.execute('SET CHARACTER SET utf8;')
    cur.execute('SET character_set_connection=utf8;')
    return cur
conn = get_conn(db_info)
cur = init_cur(conn)

In [3]:
query = """SELECT user_name From notable"""
df_notable = pd.read_sql_query (query, conn)

In [4]:
query = """SELECT user_name, bottle_name, brewer, review, overall From ratings"""
df = pd.read_sql_query (query, conn)

In [5]:
df["bottle"] = df["brewer"] + ' ' + df["bottle_name"]

In [6]:
df.head()

,user_name,bottle_name,brewer,review,overall,bottle
0,--Dom--,2XIPA,Southern Tier Brewing Company,Poured from a 12 oz bottle into a snifter. Pou...,3.75,Southern Tier Brewing Company 2XIPA
1,--Dom--,60 Minute IPA,Dogfish Head Craft Brewery,Poures from a 12 oz bottle into a tulip. Pours...,4.00,Dogfish Head Craft Brewery 60 Minute IPA
2,--Dom--,90 Minute IPA,Dogfish Head Craft Brewery,Poured from a 12 oz bottle into a snifter. Pou...,4.75,Dogfish Head Craft Brewery 90 Minute IPA
3,--Dom--,Alpha King,3 Floyds Brewing Co.,A great APA. A lot more flavor here than most ...,4.25,3 Floyds Brewing Co. Alpha King
4,--Dom--,Alter Ego Black IPA,4 Hands Brewing Co.,While I tend to only like beers over 6.5ish ab...,4.00,4 Hands Brewing Co. Alter Ego Black IPA


In [7]:
print('Total Bottles:',len(df["bottle"].unique()))

Total Bottles: 38607


In [8]:
print('Total Users:',len(df["user_name"].unique()))

Total Users: 55504


In [9]:
# downsmapling
bottle = df['bottle'].value_counts()[0:2500].index
rand_user = np.random.choice(df['user_name'].unique(),5000)
df_temp = df[df['bottle'].isin(bottle)][df['user_name'].isin(rand_user)].reset_index().copy()
df_temp.drop_duplicates(inplace=True)

C:\Users\fodkastir\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


In [10]:
# get reviews for clustering
df_review = df_temp[df_temp['user_name'].isin(df_notable['user_name'])]
df_review_sum = df_review.groupby(['bottle'])['review'].sum()
df_review_sum = pd.DataFrame({'bottle':df_review_sum.index, 'bottle_label':range(0,len(df_review_sum)), 'review':df_review_sum.values} )
df_review_sum.head()

,bottle,bottle_label,review
0,21st Amendment Brewery 21st Amendment IPA,0,Pours a cloudy light copper with more than a f...
1,21st Amendment Brewery Back In Black,1,Basic black with a finger of head that left be...
2,21st Amendment Brewery Fireside Chat,2,Brownish copper with a half finger of head. Mi...
3,21st Amendment Brewery Hell Or High Watermelon...,3,Pours a pale straw color with a thin film of h...
4,21st Amendment Brewery Hop Crisis,4,Pours a hazy straw color with a finger of head...


In [11]:
le_user = preprocessing.LabelEncoder()
le_user.fit(df_temp['user_name'])
df_temp['user_label'] = le_user.transform(df_temp['user_name']) 

df_temp = pd.merge(df_temp, df_review_sum[['bottle','bottle_label']], on='bottle')
df_temp.head()

,index,user_name,bottle_name,brewer,review,overall,bottle,user_label,bottle_label
0,235,-steve-,Tröegs Nugget Nectar,Tröegs Brewing Company,A - Red hued with generous head. Lacy foam sti...,4.0,Tröegs Brewing Company Tröegs Nugget Nectar,0,2305
1,3115,2BDChicago,Tröegs Nugget Nectar,Tröegs Brewing Company,"A: Clear, rich golden amber. Delicate white fo...",4.5,Tröegs Brewing Company Tröegs Nugget Nectar,12,2305
2,7831,57md,Tröegs Nugget Nectar,Tröegs Brewing Company,Appearance is a striking copper with half a fi...,4.5,Tröegs Brewing Company Tröegs Nugget Nectar,21,2305
3,11281,aaron,Tröegs Nugget Nectar,Tröegs Brewing Company,This one is really delicious -- pairing a citr...,3.5,Tröegs Brewing Company Tröegs Nugget Nectar,1981,2305
4,14977,ABOWMan,Tröegs Nugget Nectar,Tröegs Brewing Company,"Poured from the bottle, this beer is amber in ...",4.0,Tröegs Brewing Company Tröegs Nugget Nectar,84,2305


In [12]:
# create tokenizer
nlp = spacy.load('en')

def spacy_tokenize (raw_text):

    stops = set(stopwords.words("english"))

    text = BeautifulSoup(raw_text, "lxml").get_text()
       
    tokens = [token.lemma_ for token in nlp(raw_text) if not token.lemma_ in stops and len(token.lemma_) > 1 and token.lemma_ != '-PRON-']
    tokens = [token for token in tokens if re.match("^[a-zA-Z0-9_]*$", token)]
    
    return tokens

In [13]:
print('Number of Bottles:',len(df_review_sum))

Number of Bottles: 2479


In [14]:
# tokenize reviews
clean_reviews = []
for review in tqdm(df_review_sum['review']):
    clean_reviews.append(spacy_tokenize (review))

100%|██████████████████████████████████████| 2479/2479 [03:44<00:00, 11.04it/s]


In [60]:
# reviews to tfidf to lsi to kmeans labels 
def reviews_tolsi(reviews, n_topics=100):
    dic = corpora.Dictionary(reviews)
    corpus = [dic.doc2bow(text) for text in clean_reviews]
    tfidf = gensim.models.tfidfmodel.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    lsi = gensim.models.lsimodel.LsiModel(corpus_tfidf, num_topics=n_topics, id2word=dic)
    
    review_lsi = np.zeros((len(df_review_sum), n_topics), dtype="float32")
    counter = 0
    for tf in corpus_tfidf:
        review_lsi[counter] = [x[1] for x in lsi[tf]]
        counter += 1
    return review_lsi

def reviews_tolda(reviews, n_topics=100):
    dic = corpora.Dictionary(reviews)
    corpus = [dic.doc2bow(text) for text in clean_reviews]
    tfidf = gensim.models.tfidfmodel.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    lda = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics=n_topics, id2word=dic)
    
    review_lda = np.zeros((len(df_review_sum), n_topics), dtype="float32")
    counter = 0
    for tf in corpus_tfidf:
        review_lda[counter] = [x[1] for x in lda.get_document_topics(tf, minimum_probability=0)]
        counter += 1
    return review_lda

In [119]:
n_topics=100
dic = corpora.Dictionary(clean_reviews)
corpus = [dic.doc2bow(text) for text in clean_reviews]
tfidf = gensim.models.tfidfmodel.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
lsi = gensim.models.lsimodel.LsiModel(corpus_tfidf, num_topics=n_topics, id2word=dic)
lda = gensim.models.ldamodel.LdaModel(corpus_tfidf, num_topics=n_topics, id2word=dic)

In [123]:
lsi.print_topics(10)

[(0,
  '0.213*"chocolate" + 0.191*"coffee" + 0.118*"brown" + 0.114*"pine" + 0.113*"stout" + 0.110*"roasted" + 0.108*"dark" + 0.107*"black" + 0.107*"ipa" + 0.098*"grapefruit"'),
 (1,
  '-0.412*"coffee" + -0.388*"chocolate" + -0.235*"stout" + -0.186*"black" + -0.182*"roasted" + -0.153*"bourbon" + 0.149*"ipa" + 0.141*"grapefruit" + 0.140*"pine" + -0.136*"roast"'),
 (2,
  '-0.264*"pine" + -0.255*"ipa" + -0.217*"grapefruit" + 0.196*"sour" + 0.175*"cherry" + 0.175*"wheat" + 0.170*"banana" + 0.166*"pumpkin" + 0.151*"clove" + 0.149*"bourbon"'),
 (3,
  '-0.561*"bourbon" + -0.237*"oak" + 0.217*"wheat" + -0.213*"barrel" + 0.185*"coffee" + -0.181*"vanilla" + 0.180*"banana" + 0.140*"clove" + -0.122*"cherry" + 0.114*"porter"'),
 (4,
  '0.860*"pumpkin" + 0.207*"pie" + 0.180*"cinnamon" + 0.177*"nutmeg" + -0.104*"wheat" + 0.099*"spice" + -0.083*"sour" + -0.079*"banana" + 0.078*"ginger" + 0.070*"allspice"'),
 (5,
  '0.500*"bourbon" + 0.289*"wheat" + -0.270*"sour" + 0.261*"banana" + -0.229*"cherry" + -0.

In [125]:
lda.print_topics(10)

[(56,
  '0.005*"compete" + 0.004*"coffee" + 0.004*"roast" + 0.004*"ghost" + 0.004*"roasted" + 0.003*"chocolate" + 0.003*"pils" + 0.003*"piny" + 0.003*"decidedly" + 0.003*"necessarily"'),
 (42,
  '0.007*"lost" + 0.004*"speakeasy" + 0.004*"glug" + 0.004*"graininess" + 0.003*"jab" + 0.003*"brewers" + 0.003*"swampy" + 0.002*"beech" + 0.002*"ohhh" + 0.002*"flourescent"'),
 (39,
  '0.007*"advertised" + 0.004*"overactive" + 0.003*"furious" + 0.003*"digestable" + 0.003*"cm" + 0.003*"deny" + 0.002*"fate" + 0.002*"toats" + 0.002*"gallatin" + 0.002*"tn"'),
 (4,
  '0.007*"coffee" + 0.007*"chocolate" + 0.005*"brown" + 0.005*"cherry" + 0.004*"dark" + 0.004*"roasted" + 0.003*"berry" + 0.003*"black" + 0.003*"oak" + 0.003*"roast"'),
 (35,
  '0.007*"octoberfest" + 0.004*"chill" + 0.004*"mixer" + 0.003*"fennel" + 0.003*"impressively" + 0.003*"jul" + 0.003*"filmishmish" + 0.003*"protien" + 0.002*"blanced" + 0.002*"encompass"'),
 (75,
  '0.007*"microbrewery" + 0.004*"drafting" + 0.004*"lathering" + 0.002*"

In [116]:
review_lsi = reviews_tolsi(clean_reviews)
sim_matrix_lsi = cosine_similarity(review_lsi, review_lsi)

In [23]:
train_ind = np.random.choice(range(len(df_temp)),int(len(df_temp)*0.8))
test_ind = np.setdiff1d(range(len(df_temp)), train_ind)
df_temp_train = df_temp.ix[train_ind] 
df_temp_test = df_temp.ix[test_ind] 

df_temp_train.drop_duplicates(inplace=True)
df_temp_test.drop_duplicates(inplace=True)

In [33]:
df_temp_test.head()

,index,user_name,bottle_name,brewer,review,overall,bottle,user_label,bottle_label
0,235,-steve-,Tröegs Nugget Nectar,Tröegs Brewing Company,A - Red hued with generous head. Lacy foam sti...,4.0,Tröegs Brewing Company Tröegs Nugget Nectar,0,2305
4,14977,ABOWMan,Tröegs Nugget Nectar,Tröegs Brewing Company,"Poured from the bottle, this beer is amber in ...",4.0,Tröegs Brewing Company Tröegs Nugget Nectar,84,2305
5,26993,aerozeppl,Tröegs Nugget Nectar,Tröegs Brewing Company,12oz bottle into a pint glass.<br />\n<br />\n...,4.5,Tröegs Brewing Company Tröegs Nugget Nectar,2007,2305
6,35836,AKBelgianBeast,Tröegs Nugget Nectar,Tröegs Brewing Company,Thanks to franksnbeans for the trade. <br />\n...,4.5,Tröegs Brewing Company Tröegs Nugget Nectar,91,2305
9,68865,AnPiobaire,Tröegs Nugget Nectar,Tröegs Brewing Company,"Poured from a 12 oz bottle, into a goblet. It ...",4.5,Tröegs Brewing Company Tröegs Nugget Nectar,136,2305


In [34]:
rating_mat = coo_matrix((df_temp_train['overall'],(df_temp_train['bottle_label'], df_temp_train['user_label']))).toarray()

In [115]:
def KNN (row, k=1, sim_matrix=sim_matrix, rating_mat=rating_mat):
    i = int(row['user_label'])
    j = int(row['bottle_label'])
    top_k = []
    a = sim_matrix[j].argsort()[::-1]
    b = rating_mat[:,i].nonzero()[0]
    for item in a:
        if item in b:
            top_k.append(item)
        if len(top_k) == k:
            break
    sim = sim_matrix[j][top_k]
    rat = rating_mat[top_k,i]
    deno = sum(abs(sim))
    num = np.dot(rat,sim)
    return num/deno 

In [117]:
df_temp_test['K1'] = df_temp_test.apply(KNN ,args=(1,sim_matrix_lsi),axis=1)
df_temp_test['K5'] = df_temp_test.apply(KNN ,args=(5,sim_matrix_lsi),axis=1)
df_temp_test['K10'] = df_temp_test.apply(KNN ,args=(10,sim_matrix_lsi),axis=1)
df_temp_test['K20'] = df_temp_test.apply(KNN ,args=(20,sim_matrix_lsi),axis=1)
df_temp_test['K30'] = df_temp_test.apply(KNN ,args=(30,sim_matrix_lsi),axis=1)
df_temp_test['K40'] = df_temp_test.apply(KNN ,args=(40,sim_matrix_lsi),axis=1)
df_temp_test['K50'] = df_temp_test.apply(KNN ,args=(50,sim_matrix_lsi),axis=1)

In [118]:
print('RMSE of K=1:',np.sqrt(((df_temp_test['overall'] - df_temp_test['K1'])**2).mean()))
print('RMSE of K=5:',np.sqrt(((df_temp_test['overall'] - df_temp_test['K5'])**2).mean()))
print('RMSE of K=10:',np.sqrt(((df_temp_test['overall'] - df_temp_test['K10'])**2).mean()))
print('RMSE of K=20:',np.sqrt(((df_temp_test['overall'] - df_temp_test['K20'])**2).mean()))
print('RMSE of K=30:',np.sqrt(((df_temp_test['overall'] - df_temp_test['K30'])**2).mean()))
print('RMSE of K=40:',np.sqrt(((df_temp_test['overall'] - df_temp_test['K40'])**2).mean()))
print('RMSE of K=50:',np.sqrt(((df_temp_test['overall'] - df_temp_test['K50'])**2).mean()))

RMSE of K=1: 0.783711713479
RMSE of K=5: 0.636650835102
RMSE of K=10: 0.622786310201
RMSE of K=20: 0.618608015686
RMSE of K=30: 0.618953160199
RMSE of K=40: 0.619923938829
RMSE of K=50: 0.620894338044


In [53]:
print('MAE of K=1:',abs((df_temp_test['overall'] - df_temp_test['K1'])).mean())
print('MAE of K=5:',abs((df_temp_test['overall'] - df_temp_test['K5'])).mean())
print('MAE of K=10:',abs((df_temp_test['overall'] - df_temp_test['K10'])).mean())
print('MAE of K=20:',abs((df_temp_test['overall'] - df_temp_test['K20'])).mean())
print('MAE of K=30:',abs((df_temp_test['overall'] - df_temp_test['K30'])).mean())
print('MAE of K=40:',abs((df_temp_test['overall'] - df_temp_test['K40'])).mean())
print('MAE of K=50:',abs((df_temp_test['overall'] - df_temp_test['K50'])).mean())

MAE of K=1: 0.550429906542056
MAE of K=5: 0.4610888125817281
MAE of K=10: 0.44861643063503137
MAE of K=20: 0.444136464229022
MAE of K=30: 0.4437259351320703
MAE of K=40: 0.4441832893234423
MAE of K=50: 0.44486833345267784


In [61]:
review_lda = reviews_tolda(clean_reviews)
sim_matrix_lda = cosine_similarity(review_lda, review_lda)

In [63]:
df_temp_test_lda = df_temp_test.copy()

In [64]:
df_temp_test_lda['K1'] = df_temp_test_lda.apply(KNN ,args=(1,),axis=1)
df_temp_test_lda['K5'] = df_temp_test_lda.apply(KNN ,args=(5,),axis=1)
df_temp_test_lda['K10'] = df_temp_test_lda.apply(KNN ,args=(10,),axis=1)
df_temp_test_lda['K20'] = df_temp_test_lda.apply(KNN ,args=(20,),axis=1)
df_temp_test_lda['K30'] = df_temp_test_lda.apply(KNN ,args=(30,),axis=1)
df_temp_test_lda['K40'] = df_temp_test_lda.apply(KNN ,args=(40,),axis=1)
df_temp_test_lda['K50'] = df_temp_test_lda.apply(KNN ,args=(50,),axis=1)

In [65]:
print('RMSE of K=1:',np.sqrt(((df_temp_test_lda['overall'] - df_temp_test_lda['K1'])**2).mean()))
print('RMSE of K=5:',np.sqrt(((df_temp_test_lda['overall'] - df_temp_test_lda['K5'])**2).mean()))
print('RMSE of K=10:',np.sqrt(((df_temp_test_lda['overall'] - df_temp_test_lda['K10'])**2).mean()))
print('RMSE of K=20:',np.sqrt(((df_temp_test_lda['overall'] - df_temp_test_lda['K20'])**2).mean()))
print('RMSE of K=30:',np.sqrt(((df_temp_test_lda['overall'] - df_temp_test_lda['K30'])**2).mean()))
print('RMSE of K=40:',np.sqrt(((df_temp_test_lda['overall'] - df_temp_test_lda['K40'])**2).mean()))
print('RMSE of K=50:',np.sqrt(((df_temp_test_lda['overall'] - df_temp_test_lda['K50'])**2).mean()))

RMSE of K=1: 0.828088479328
RMSE of K=5: 0.670906769776
RMSE of K=10: 0.652095788004
RMSE of K=20: 0.644506919107
RMSE of K=30: 0.642449671665
RMSE of K=40: 0.641792545387
RMSE of K=50: 0.641603847156


In [66]:
print('MAE of K=1:',abs((df_temp_test_lda['overall'] - df_temp_test_lda['K1'])).mean())
print('MAE of K=5:',abs((df_temp_test_lda['overall'] - df_temp_test_lda['K5'])).mean())
print('MAE of K=10:',abs((df_temp_test_lda['overall'] - df_temp_test_lda['K10'])).mean())
print('MAE of K=20:',abs((df_temp_test_lda['overall'] - df_temp_test_lda['K20'])).mean())
print('MAE of K=30:',abs((df_temp_test_lda['overall'] - df_temp_test_lda['K30'])).mean())
print('MAE of K=40:',abs((df_temp_test_lda['overall'] - df_temp_test_lda['K40'])).mean())
print('MAE of K=50:',abs((df_temp_test_lda['overall'] - df_temp_test_lda['K50'])).mean())

MAE of K=1: 0.5904485981308412
MAE of K=5: 0.4870090036915699
MAE of K=10: 0.47051404564998933
MAE of K=20: 0.4632370140800693
MAE of K=30: 0.4609535361148403
MAE of K=40: 0.460560043902872
MAE of K=50: 0.4603499568402314


In [79]:
sim_matrix_corr_lsi = np.corrcoef(review_lsi)

In [80]:
df_temp_test['K1_corr'] = df_temp_test.apply(KNN ,args=(1,sim_matrix_corr_lsi),axis=1)
df_temp_test['K5_corr'] = df_temp_test.apply(KNN ,args=(5,sim_matrix_corr_lsi),axis=1)
df_temp_test['K10_corr'] = df_temp_test.apply(KNN ,args=(10,sim_matrix_corr_lsi),axis=1)
df_temp_test['K20_corr'] = df_temp_test.apply(KNN ,args=(20,sim_matrix_corr_lsi),axis=1)
df_temp_test['K30_corr'] = df_temp_test.apply(KNN ,args=(30,sim_matrix_corr_lsi),axis=1)
df_temp_test['K40_corr'] = df_temp_test.apply(KNN ,args=(40,sim_matrix_corr_lsi),axis=1)
df_temp_test['K50_corr'] = df_temp_test.apply(KNN ,args=(50,sim_matrix_corr_lsi),axis=1)

In [81]:
print('RMSE of K=1:',np.sqrt(((df_temp_test['overall'] - df_temp_test['K1_corr'])**2).mean()))
print('RMSE of K=5:',np.sqrt(((df_temp_test['overall'] - df_temp_test['K5_corr'])**2).mean()))
print('RMSE of K=10:',np.sqrt(((df_temp_test['overall'] - df_temp_test['K10_corr'])**2).mean()))
print('RMSE of K=20:',np.sqrt(((df_temp_test['overall'] - df_temp_test['K20_corr'])**2).mean()))
print('RMSE of K=30:',np.sqrt(((df_temp_test['overall'] - df_temp_test['K30_corr'])**2).mean()))
print('RMSE of K=40:',np.sqrt(((df_temp_test['overall'] - df_temp_test['K40_corr'])**2).mean()))
print('RMSE of K=50:',np.sqrt(((df_temp_test['overall'] - df_temp_test['K50_corr'])**2).mean()))

RMSE of K=1: 0.783425460215
RMSE of K=5: 0.637196107702
RMSE of K=10: 0.622587251318
RMSE of K=20: 0.618492821784
RMSE of K=30: 0.61890879194
RMSE of K=40: 0.619718214387
RMSE of K=50: 0.620760987385


In [82]:
print('MAE of K=1:',abs((df_temp_test['overall'] - df_temp_test['K1_corr'])).mean())
print('MAE of K=5:',abs((df_temp_test['overall'] - df_temp_test['K5_corr'])).mean())
print('MAE of K=10:',abs((df_temp_test['overall'] - df_temp_test['K10_corr'])).mean())
print('MAE of K=20:',abs((df_temp_test['overall'] - df_temp_test['K20_corr'])).mean())
print('MAE of K=30:',abs((df_temp_test['overall'] - df_temp_test['K30_corr'])).mean())
print('MAE of K=40:',abs((df_temp_test['overall'] - df_temp_test['K40_corr'])).mean())
print('MAE of K=50:',abs((df_temp_test['overall'] - df_temp_test['K50_corr'])).mean())

MAE of K=1: 0.5501495327102803
MAE of K=5: 0.46109549223830204
MAE of K=10: 0.44838131000121384
MAE of K=20: 0.44392615465790297
MAE of K=30: 0.44353638479661056
MAE of K=40: 0.4440856481152861
MAE of K=50: 0.4447119927200129


In [83]:
sim_matrix_corr_lda = np.corrcoef(review_lda)

In [84]:
df_temp_test_lda['K1_corr'] = df_temp_test_lda.apply(KNN ,args=(1,sim_matrix_corr_lda),axis=1)
df_temp_test_lda['K5_corr'] = df_temp_test_lda.apply(KNN ,args=(5,sim_matrix_corr_lda),axis=1)
df_temp_test_lda['K10_corr'] = df_temp_test_lda.apply(KNN ,args=(10,sim_matrix_corr_lda),axis=1)
df_temp_test_lda['K20_corr'] = df_temp_test_lda.apply(KNN ,args=(20,sim_matrix_corr_lda),axis=1)
df_temp_test_lda['K30_corr'] = df_temp_test_lda.apply(KNN ,args=(30,sim_matrix_corr_lda),axis=1)
df_temp_test_lda['K40_corr'] = df_temp_test_lda.apply(KNN ,args=(40,sim_matrix_corr_lda),axis=1)
df_temp_test_lda['K50_corr'] = df_temp_test_lda.apply(KNN ,args=(50,sim_matrix_corr_lda),axis=1)

In [85]:
print('RMSE of K=1:',np.sqrt(((df_temp_test_lda['overall'] - df_temp_test_lda['K1_corr'])**2).mean()))
print('RMSE of K=5:',np.sqrt(((df_temp_test_lda['overall'] - df_temp_test_lda['K5_corr'])**2).mean()))
print('RMSE of K=10:',np.sqrt(((df_temp_test_lda['overall'] - df_temp_test_lda['K10_corr'])**2).mean()))
print('RMSE of K=20:',np.sqrt(((df_temp_test_lda['overall'] - df_temp_test_lda['K20_corr'])**2).mean()))
print('RMSE of K=30:',np.sqrt(((df_temp_test_lda['overall'] - df_temp_test_lda['K30_corr'])**2).mean()))
print('RMSE of K=40:',np.sqrt(((df_temp_test_lda['overall'] - df_temp_test_lda['K40_corr'])**2).mean()))
print('RMSE of K=50:',np.sqrt(((df_temp_test_lda['overall'] - df_temp_test_lda['K50_corr'])**2).mean()))

RMSE of K=1: 0.828385623628
RMSE of K=5: 0.701476553279
RMSE of K=10: 0.684046156581
RMSE of K=20: 0.863513445966
RMSE of K=30: 0.859865564016
RMSE of K=40: 0.862228371094
RMSE of K=50: 1.00241005038


In [86]:
print('MAE of K=1:',abs((df_temp_test_lda['overall'] - df_temp_test_lda['K1_corr'])).mean())
print('MAE of K=5:',abs((df_temp_test_lda['overall'] - df_temp_test_lda['K5_corr'])).mean())
print('MAE of K=10:',abs((df_temp_test_lda['overall'] - df_temp_test_lda['K10_corr'])).mean())
print('MAE of K=20:',abs((df_temp_test_lda['overall'] - df_temp_test_lda['K20_corr'])).mean())
print('MAE of K=30:',abs((df_temp_test_lda['overall'] - df_temp_test_lda['K30_corr'])).mean())
print('MAE of K=40:',abs((df_temp_test_lda['overall'] - df_temp_test_lda['K40_corr'])).mean())
print('MAE of K=50:',abs((df_temp_test_lda['overall'] - df_temp_test_lda['K50_corr'])).mean())

MAE of K=1: 0.590386292834891
MAE of K=5: 0.4886022782957587
MAE of K=10: 0.4723539213505411
MAE of K=20: 0.46795871596938377
MAE of K=30: 0.4654838677265959
MAE of K=40: 0.46561910773038057
MAE of K=50: 0.46890076173758366


In [128]:
df_temp_train[['user_name','bottle_name','brewer','bottle','user_label','bottle_label','overall']].to_csv('train.csv')
df_temp_test[['user_name','bottle_name','brewer','bottle','user_label','bottle_label','overall']].to_csv('test.csv')